[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elsamuel27/snapthon/blob/main/Snapthon.ipynb)

# Instalaciones

In [ ]:
!pip install -U pip

In [ ]:
!pip install pytube

In [ ]:
!python -m pip install --upgrade pytube

In [ ]:
!sudo apt install zip

# Importaciones

In [ ]:
from pytube import YouTube
from pytube import Playlist
from google.colab import files
import os

# Declarar variables

In [ ]:
print("1 para vídeo, 2 para playlist, 3 para varios vídeos")
mult = input()
if mult == "3":
  URLs = []
  i = 0
  o = int(input("¿Cuántos son?: "))
  while i < o:
    link = input("Enlace: ")
    URLs.append(link)
    i += 1
else:
  link = input("Pega su enlace: ")

print("1 para vídeo, 2 para audio")
ft = input()
format = input("¿Usar formatos mp4 y mp3? [s/n]: ")

In [ ]:
if format == "s":
    if ft == "1":
        format2 = "mp4"
    elif ft == "2":
        format2 = "mp3"
elif format == "n":
    format2 = input("Entonces introduce el formato que quieras, en minúscula y sin punto: ")

In [ ]:
if mult == "2" or "3":
  route = "/content/Descargados"
else:
  route = "/content"

# Descargar

In [ ]:
if mult == "1":
    yt = YouTube(link)
    title = yt.title.replace("\"", "")
    title = title.replace("/", "-")
    file = f"{title}.{format2}"

    if ft == "1":
        yd = yt.streams.get_highest_resolution()

        yd.download(route, file)
        print("¡Vídeo descargado! Hasta luego :D")
    elif ft == "2":
        yd = yt.streams.get_audio_only()

        yd.download(route, file)
        print("¡Vídeo descargado! Hasta luego :D")
elif mult == "2":
    pl = Playlist(link)
    if ft == "1":
        for video in pl.videos:
            title = yt.title.replace("\"", "")
            title = title.replace("/", "-")
            file = f"{title}.{format2}"
            try:
                yd = video.streams.get_highest_resolution()
                yd.download(route, file)
                print("+1 vídeo en la carpeta")
            except:
                print("No se puede acceder a los datos de este vídeo. Saltamos al siguiente")
    elif ft == "2":
        for video in pl.videos:
            title = video.title.replace("\"", "")
            title = title.replace("/", "-")
            try:
                yd = video.streams.get_audio_only()
                yd.download(route, f"{title}.{format2}")
                print("+1 vídeo en la carpeta")
            except:
                print("No se puede acceder a los datos de este vídeo. Saltamos al siguiente")
elif mult == "3":
  for video in URLs:
    yt = YouTube(video)
    title = yt.title.replace("\"", "")
    title = title.replace("/", "-")
    file = f"{title}.{format2}"
    if ft == "1":
      try:
        yd = yt.streams.get_highest_resolution()
        yd.download(route, file)
        print("+1 vídeo en la carpeta")
      except:
        print("No se puede acceder a los datos de este vídeo. Saltamos al siguiente")
    elif ft == "2":
      try:
        yd = yt.streams.get_audio_only()
        yd.download(route, file)
        print("+1 vídeo en la carpeta")
      except:
        print("No se puede acceder a los datos de este vídeo. Saltamos al siguiente")

# Descargar (En tu PC)

In [ ]:
total = os.listdir(route)
total = len(total)
if total > 1:
  print(f"Has descargado un total de {total} vídeos")
  print("¿Seguro que quieres descargar todos de golpe?")
else:
  print("¿Quieres descargar tu vídeo?")

confirm = input("[s/n]: ")

In [ ]:
if confirm == "s":
  if mult == "2" or "3":
    for i in os.listdir(route):
      files.download('/content/Descargados/'+i)
  else:
    files.download(f"{route}/{title}.{format2}")
else:
  print("Entonces nada XD")